# Load Dataset

abalone dataset

In [1]:
from libsvm.svmutil import *

In [6]:
y, x = svm_read_problem('./cached_datasets/abalone/abalone.data')

ValueError: could not convert string to float: 'M,0.455,0.365,0.095,0.514,0.2245,0.101,0.15,15'

In [8]:
import pandas as pd

In [10]:
data = pd.read_csv("./cached_datasets/abalone/abalone.data", header=None)

In [11]:
data[0] = data[0].map(lambda x: 1 if x == 'M' else -1)

In [12]:
data['label'] = (data[8]>9).astype(int)

In [13]:
data

,0,1,2,3,4,5,6,7,8,label
0,1,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15,1
1,1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7,0
2,-1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9,0
3,1,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10,1
4,-1,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7,0
...,...,...,...,...,...,...,...,...,...,...
4172,-1,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11,1
4173,1,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10,1
4174,1,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9,0
4175,-1,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10,1


In [16]:
'''
Author: Xiang Pan
Date: 2022-03-08 03:58:47
LastEditTime: 2022-03-08 15:15:59
LastEditors: Xiang Pan
Description: 
FilePath: /HW2/c.ipynb
@email: xiangpan@nyu.edu
'''
train_size = 3133
test_size = 1044
train_data = data.iloc[:train_size, :]
test_data = data.iloc[train_size:, :]

In [17]:
len(train_data)

3133

In [18]:
len(test_data)

1044

In [28]:
train_X = train_data.iloc[:, :8]
train_Y = train_data.iloc[:, 9]

# LibSVM

In [30]:
y, x = svm_read_problem('./cached_datasets/abalone/abalone_scale', return_scipy = True)

## train_code

In [33]:
c = 3
m = svm_train(y[:train_size], x[:train_size, :], f'-c {c}')

*
optimization finished, #iter = 84
nu = 0.247241
obj = -246.739771, rho = 0.580663
nSV = 97, nBSV = 93
*
optimization finished, #iter = 373
nu = 0.257095
obj = -449.512094, rho = 0.577862
nSV = 162, nBSV = 149
*.
*
optimization finished, #iter = 852
nu = 0.278986
obj = -461.552618, rho = 0.933478
nSV = 167, nBSV = 145
*
optimization finished, #iter = 172
nu = 0.291325
obj = -396.565224, rho = 0.196719
nSV = 151, nBSV = 143
.
*.*
optimization finished, #iter = 206
nu = 0.428571
obj = -125.158098, rho = -0.871855
nSV = 47, nBSV = 37
*
optimization finished, #iter = 120
nu = 0.737705
obj = -263.883487, rho = -0.467768
nSV = 94, nBSV = 87
.
*...*
optimization finished, #iter = 443
nu = 0.475248
obj = -143.960493, rho = -0.987776
nSV = 59, nBSV = 35
*.*
optimization finished, #iter = 173
nu = 0.905882
obj = -456.634291, rho = 0.764061
nSV = 158, nBSV = 149
*.*
optimization finished, #iter = 532
nu = 0.363208
obj = -460.540406, rho = 0.962663
nSV = 162, nBSV = 147
.
*.
*
optimization finish

In [37]:
C

[0.00411522633744856,
 0.012345679012345678,
 0.037037037037037035,
 0.1111111111111111,
 0.3333333333333333,
 1.0,
 3.0,
 9.0,
 27.0,
 81.0]

In [48]:
C = [math.pow(3,i) for i in range(-10, 10)]
p_label_list = []
p_acc_list = []
p_val_list = []
for c in C:
    # polynomial kernel
    m = svm_train(y[:train_size], x[:train_size, :], f'-c {c} -t 1 -d 3 -v 5')
    # train_acc
    p_label, p_acc, p_val = svm_predict(y[train_size:], x[train_size:, :], m)
    p_label_list.append(p_label)
    p_acc_list.append(p_acc)
    p_val_list.append(p_val)
    print(f'C: {c}, accuracy: {p_acc[0]}')

*
optimization finished, #iter = 62
nu = 0.400000
obj = -0.002100, rho = 0.999983
nSV = 124, nBSV = 124
*
optimization finished, #iter = 62
nu = 0.258333
obj = -0.002100, rho = 1.000000
nSV = 124, nBSV = 124
*
optimization finished, #iter = 62
nu = 0.280543
obj = -0.002100, rho = 1.000000
nSV = 124, nBSV = 124
*
optimization finished, #iter = 62
nu = 0.306173
obj = -0.002100, rho = 0.999999
nSV = 124, nBSV = 124
*
optimization finished, #iter = 17
nu = 0.430380
obj = -0.000576, rho = -1.000000
nSV = 34, nBSV = 34
*
optimization finished, #iter = 36
nu = 0.734694
obj = -0.001219, rho = -0.999999
nSV = 72, nBSV = 72
*
optimization finished, #iter = 20
nu = 0.487805
obj = -0.000677, rho = -1.000000
nSV = 40, nBSV = 40
*
optimization finished, #iter = 62
nu = 0.905109
obj = -0.002100, rho = 1.000000
nSV = 124, nBSV = 124
*
optimization finished, #iter = 62
nu = 0.364706
obj = -0.002100, rho = 1.000000
nSV = 124, nBSV = 124
*
optimization finished, #iter = 62
nu = 0.558559
obj = -0.002100, 

AttributeError: 'float' object has no attribute 'get_svm_type'

In [47]:
p_acc_list

[(15.996168582375478, 10.39367816091954, nan),
 (15.996168582375478, 10.39367816091954, nan),
 (15.996168582375478, 10.39367816091954, nan),
 (15.996168582375478, 10.39367816091954, nan),
 (15.996168582375478, 10.39367816091954, nan),
 (15.996168582375478, 10.39367816091954, nan),
 (15.996168582375478, 10.39367816091954, nan),
 (16.47509578544061, 9.830459770114942, 0.12940498618932342),
 (18.582375478927204, 9.563218390804598, 0.24153870596790292),
 (21.26436781609195, 9.441570881226054, 0.29443587900165274),
 (24.042145593869733, 7.813218390804598, 0.3372396547455698),
 (24.71264367816092, 7.751915708812261, 0.339588686295031),
 (24.808429118773947, 7.708812260536399, 0.3567526351105587),
 (26.053639846743295, 7.436781609195402, 0.3780135296573146),
 (27.107279693486593, 7.293103448275862, 0.38612528407203844),
 (27.298850574712645, 6.842911877394636, 0.4209378988064059),
 (27.490421455938694, 6.282567049808429, 0.44582058155588616),
 (27.873563218390807, 5.946360153256705, 0.4607282

# Split Train Dataset into five parts

In [14]:
import math
part_size = math.floor(3133/5)

In [35]:
part_list = []
# shuffle train data
train_data = train_data.sample(frac=1)
for i in range(5):
    part_list.append(train_data.iloc[i*part_size:(i+1)*part_size, :])

In [39]:
for i in range(5):
    print(len(part_list[i]))

626
626
626
626
626


# polynomial kernels libsvm

In [50]:
from libsvm.svmutil import *

## train model

In [52]:
idx = 0
train_X = part_list[idx].iloc[:, :8].to_numpy()
train_Y = part_list[idx].iloc[:, 8].to_numpy()

In [55]:
train_X.to_numpy()

array([['M', 0.59, 0.47, ..., 0.479, 0.2425, 0.309],
       ['F', 0.5, 0.4, ..., 0.2395, 0.126, 0.185],
       ['M', 0.27, 0.2, ..., 0.0465, 0.028, 0.04],
       ...,
       ['I', 0.555, 0.46, ..., 0.3845, 0.158, 0.2765],
       ['M', 0.635, 0.5, ..., 0.4405, 0.2315, 0.387],
       ['F', 0.535, 0.445, ..., 0.417, 0.199, 0.24]], dtype=object)

In [51]:
prob = svm_problem(train_Y,train_X)